In [1]:
import os
import re

import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix, classification_report

In [2]:
DATA_PATH = "data/outputs/"

In [3]:
predictions = {file[:-4]: pd.read_csv(DATA_PATH+file) for file in os.listdir(DATA_PATH)}

In [4]:
test_df = predictions.pop('test_df')
test_df

,Unnamed: 0,id_pat,date,activity,timeline,Acc_x,Acc_y,Acc_z,Gyro_x,Gyro_y,Gyro_z,appui_leve,foot_type
0,1081748,P42,2020-07-13,1minJoint,0.000000,5.66,-7.96,0.79,0.074176,-0.206167,-0.014181,0.0,0
1,1081749,P42,2020-07-13,1minJoint,0.025316,5.76,-7.59,0.95,0.070904,-0.077449,-0.034907,0.0,0
2,1081750,P42,2020-07-13,1minJoint,0.050633,5.82,-7.62,1.08,0.032725,0.068722,-0.005454,0.0,0
3,1081751,P42,2020-07-13,1minJoint,0.075949,6.10,-7.73,1.18,-0.061087,0.297797,0.014181,0.0,0
4,1081752,P42,2020-07-13,1minJoint,0.101266,5.78,-7.69,1.28,-0.215984,1.015563,0.075267,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
55663,2219159,P42,2020-07-13,Step,27.022901,1.14,-9.64,1.67,0.112356,-0.186532,0.010908,0.0,1
55664,2219160,P42,2020-07-13,Step,27.048346,1.09,-9.58,1.72,0.267254,-0.296706,0.047997,0.0,1
55665,2219161,P42,2020-07-13,Step,27.073791,1.31,-9.28,1.66,0.234529,-0.265072,0.066541,0.0,1
55666,2219162,P42,2020-07-13,Step,27.099237,0.99,-9.48,1.78,0.266163,-0.250891,0.105811,0.0,1


In [5]:
def evaluate(y_real, y_pred):
    print("Accuracy score: ", accuracy_score(y_real, y_pred))
    print("Confusion matrix: \n", confusion_matrix(y_real, y_pred))
    print("Classification report: \n", classification_report(y_real, y_pred))

    plt.figure(figsize=(20, 5))
    plt.plot(y_real, label='Real')
    plt.plot(y_pred, label='Predictions')
    plt.legend()
    plt.figure(figsize=(20, 5))
    plt.plot(y_real)
    plt.figure(figsize=(20, 5))
    plt.plot(y_pred)
    plt.show()

    fig = go.Figure()
    fig.add_trace(go.Scatter(y=y_real, name='Real values'))
    fig.add_trace(go.Scatter(y=y_pred, name='Predicted values'))
    pio.show(fig)

def evaluate_models(test_df, model_preds, model_name):
    """Evaualtes the predictions made by the models by reformatting them into their initial length
    """
    window_size = int(re.search(r'_ws_(.*?)_transform', model_name).group(1))
    y_real_test_pred = model_preds.predictions
    y_real_test = test_df.appui_leve.values[:len(y_real_test_pred)]
    print(
        """
        ############################################
        {} evaluation...
        ############################################
        """.format(model_name)
    )
    print("########## All test data ##########")
    #self.evaluate(y_real_test_pred, y_real_test)
    acc = accuracy_score(y_real_test, y_real_test_pred)
    f1 = f1_score(y_real_test, y_real_test_pred)
    rec = recall_score(y_real_test, y_real_test_pred)
    pre = precision_score(y_real_test, y_real_test_pred)
    print("Accuracy score: ", acc)
    print("F1 score score: ", f1)
    print("Recall: ", rec)
    print("Precision: ", pre)
    print("Confusion matrix: \n", confusion_matrix(y_real_test, y_real_test_pred))
    print("Classification report: \n", classification_report(y_real_test, y_real_test_pred))
    tn, fp, fn, tp = confusion_matrix(y_real_test, y_real_test_pred).ravel()
    spe = tn / (tn+fp)
    sen = tp / (tp+fn)
    print("Specificity: ", spe)
    print("Sensitivity: ", sen)
    print("TP; ", tp)
    print("FP; ", fp)
    print("TN; ", tn)
    print("FN; ", fn)
    return model_name, acc, f1, rec, pre, spe, sen, tn, fp, fn, tp

    #print("########## Left/Right data ##########")
    #self.test_df['preds'] = np.append(y_real_test_pred, np.zeros(self.test_df.shape[0] - len(y_real_test_pred)))
    #test_df_g = self.test_df[self.test_df.foot_type == 0]
    #test_df_d = self.test_df[self.test_df.foot_type == 1]
    #self.evaluate(test_df_g.appui_leve, test_df_g.preds)
    #self.evaluate(test_df_d.appui_leve, test_df_d.preds)

    #print("########## Per activity data ##########")
    #for activity in test_df_g.activity.unique():
    #    print("#### Activité: {} ####".format(activity))
    #    temp_df_g = test_df_g[test_df_g.activity == activity]
    #    temp_df_d = test_df_d[test_df_d.activity == activity]
    #    self.evaluate(temp_df_g.appui_leve, temp_df_g.preds)
    #    self.evaluate(temp_df_d.appui_leve, temp_df_d.preds)

In [6]:
all_scores = [evaluate_models(test_df, model_preds, model_name) for model_name, model_preds in predictions.items()]


        ############################################
        cnn_model_ws_5_transform_fft evaluation...
        ############################################
        
########## All test data ##########
Accuracy score:  0.9565256444803737
F1 score score:  0.9499275812124973
Recall:  0.9604602510460251
Precision:  0.9396234138354482
Confusion matrix: 
 [[30290  1475]
 [  945 22955]]
Classification report: 
               precision    recall  f1-score   support

         0.0       0.97      0.95      0.96     31765
         1.0       0.94      0.96      0.95     23900

    accuracy                           0.96     55665
   macro avg       0.95      0.96      0.96     55665
weighted avg       0.96      0.96      0.96     55665

Specificity:  0.9535652447662522
Sensitivity:  0.9604602510460251
TP;  22955
FP;  1475
TN;  30290
FN;  945

        ############################################
        cnn_model_ws_3_transform_dct evaluation...
        ###########################################

In [7]:
scores_df = pd.DataFrame(all_scores, columns = ["model_name", "acc", "f1", "rec", "pre", "spe", "sen", "tn", "fp", "fn", "tp"])
scores_df

,model_name,acc,f1,rec,pre,spe,sen,tn,fp,fn,tp
0,cnn_model_ws_5_transform_fft,0.956526,0.949928,0.960460,0.939623,0.953565,0.960460,30290,1475,945,22955
1,cnn_model_ws_3_transform_dct,0.969640,0.964820,0.969623,0.960063,0.969652,0.969623,30801,964,726,23174
2,cnn_two_channels_model_ws_6_transform_dct,0.948403,0.938398,0.915272,0.962723,0.973333,0.915272,30915,847,2025,21875
3,multi_cnn_model_ws_5_transform_None,0.960496,0.954230,0.959121,0.949389,0.961530,0.959121,30543,1222,977,22923
4,cnn_model_comp_ws_3_transform_None,0.968795,0.964201,0.978745,0.950083,0.961310,0.978745,30536,1229,508,23392
...,...,...,...,...,...,...,...,...,...,...,...
86,cnn_model_comp_ws_4_transform_None,0.965867,0.960587,0.968787,0.952526,0.963670,0.968787,30610,1154,746,23154
87,cnn_model_comp_ws_4_transform_dct,0.964573,0.958734,0.958494,0.958975,0.969147,0.958494,30784,980,992,22908
88,cnn_two_channels_model_ws_3_transform_fft,0.968239,0.963159,0.966987,0.959361,0.969180,0.966987,30786,979,789,23111
89,cnn_model_ws_7_transform_dct,0.946608,0.938094,0.942176,0.934047,0.949943,0.942176,30174,1590,1382,22518


In [8]:
extract_ws = lambda x: re.search(r'_ws_(.*?)_transform', x).group(1)
extract_transform = lambda x: re.search(r'transform_(.*)', x).group(1)
extract_model = lambda x: re.search(r'(.*?)_ws', x).group(1)

In [9]:
scores_df["window_size"] = scores_df.model_name.apply(extract_ws)
scores_df["transform"] = scores_df.model_name.apply(extract_transform)
scores_df["model"] = scores_df.model_name.apply(extract_model)

In [10]:
scores_df

,model_name,acc,f1,rec,pre,spe,sen,tn,fp,fn,tp,window_size,transform,model
0,cnn_model_ws_5_transform_fft,0.956526,0.949928,0.960460,0.939623,0.953565,0.960460,30290,1475,945,22955,5,fft,cnn_model
1,cnn_model_ws_3_transform_dct,0.969640,0.964820,0.969623,0.960063,0.969652,0.969623,30801,964,726,23174,3,dct,cnn_model
2,cnn_two_channels_model_ws_6_transform_dct,0.948403,0.938398,0.915272,0.962723,0.973333,0.915272,30915,847,2025,21875,6,dct,cnn_two_channels_model
3,multi_cnn_model_ws_5_transform_None,0.960496,0.954230,0.959121,0.949389,0.961530,0.959121,30543,1222,977,22923,5,None,multi_cnn_model
4,cnn_model_comp_ws_3_transform_None,0.968795,0.964201,0.978745,0.950083,0.961310,0.978745,30536,1229,508,23392,3,None,cnn_model_comp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,cnn_model_comp_ws_4_transform_None,0.965867,0.960587,0.968787,0.952526,0.963670,0.968787,30610,1154,746,23154,4,None,cnn_model_comp
87,cnn_model_comp_ws_4_transform_dct,0.964573,0.958734,0.958494,0.958975,0.969147,0.958494,30784,980,992,22908,4,dct,cnn_model_comp
88,cnn_two_channels_model_ws_3_transform_fft,0.968239,0.963159,0.966987,0.959361,0.969180,0.966987,30786,979,789,23111,3,fft,cnn_two_channels_model
89,cnn_model_ws_7_transform_dct,0.946608,0.938094,0.942176,0.934047,0.949943,0.942176,30174,1590,1382,22518,7,dct,cnn_model


In [11]:
del scores_df['model_name']

scores_df = scores_df[scores_df.columns[-3:].tolist() + scores_df.columns[:-3].tolist()]

In [13]:
pd.set_option('max_rows', None)

In [14]:
scores_df.sort_values(by=['window_size', 'transform', 'model']).set_index(['window_size', 'transform', 'model'])

acc        f1  \
window_size transform model                                            
3           None      cnn_model                   0.970610  0.965824   
                      cnn_model_comp              0.968795  0.964201   
                      cnn_two_channels_model      0.969173  0.964465   
                      cnn_two_channels_model_bis  0.965328  0.959758   
                      multi_cnn_model             0.969532  0.964646   
                      multi_lstm_model            0.969766  0.964919   
                      uni_lstm_model              0.966227  0.960840   
            dct       cnn_model                   0.969640  0.964820   
                      cnn_model_comp              0.953238  0.947999   
                      cnn_two_channels_model      0.965993  0.961178   
                      multi_cnn_model             0.968939  0.964077   
                      multi_lstm_model            0.970556  0.965734   
                      uni_lstm_model              0.966748  0.961705   
            fft       cnn_model                   0.968310  0.963366   
                      cnn_model_comp              0.967358  0.962566   
                      cnn_two_channels_model      0.968239  0.963159   
                      multi_cnn_model             0.967915  0.963052   
                      multi_lstm_model            0.968831  0.963975   
                      uni_lstm_model              0.966676  0.961684   
4           None      cnn_model                   0.964214  0.958141   
                      cnn_model_comp              0.965867  0.960587   
                      cnn_two_channels_model      0.965400  0.959616   
                      multi_cnn_model             0.965328  0.959392   
                      multi_lstm_model            0.964968  0.958627   
                      uni_lstm_model              0.962202  0.956276   
            dct       cnn_model                   0.964322  0.958501   
                      cnn_model_comp              0.964573  0.958734   
                      cnn_two_channels_model      0.960405  0.954732   
                      multi_cnn_model             0.963711  0.957434   
                      multi_lstm_model            0.964142  0.958134   
                      uni_lstm_model              0.961986  0.955898   
            fft       cnn_model                   0.963208  0.957105   
                      cnn_model_comp              0.963998  0.958212   
                      cnn_two_channels_model      0.963998  0.958198   
                      multi_cnn_model             0.963962  0.958247   
                      multi_lstm_model            0.962920  0.956459   
                      uni_lstm_model              0.961160  0.954770   
5           None      cnn_model                   0.959508  0.953012   
                      cnn_model_comp              0.959634  0.953694   
                      cnn_two_channels_model      0.958969  0.952496   
                      multi_cnn_model             0.960496  0.954230   
                      multi_lstm_model            0.958681  0.952193   
                      uni_lstm_model              0.955933  0.949197   
            dct       cnn_model                   0.958861  0.952132   
                      cnn_model_comp              0.957586  0.950508   
                      cnn_two_channels_model      0.957909  0.951233   
                      multi_cnn_model             0.958556  0.951872   
                      multi_lstm_model            0.959346  0.953123   
                      uni_lstm_model              0.954657  0.947937   
            fft       cnn_model                   0.956526  0.949928   
                      cnn_model_comp              0.956580  0.950129   
                      cnn_two_channels_model      0.955843  0.948983   
                      multi_cnn_model             0.956544  0.949933   
                      multi_lstm_model            0.955789  0.949388   
                      u